### Pre-sort samples
This notebook filters the model predictions.
It filters usages that have incomplete headwords and sorts the remaining usages by theire similarity to the nearest sense.

### Usage
Set the file paths in the second cell to the predictions of the models you want to filter.

In [1]:
import csv
from spacy.symbols import IS_PUNCT
import json
from nltk.corpus import wordnet as wn
import pandas as pd

In [9]:
historical_sample_file = "../data/outputs/model_predictions/sp_f3_100k_historical_unassigned_prediction_sorted_reevaluated.json"
modern_sample_file = "../data/outputs/model_predictions/SAMPLE_eng_news_2020_1M-sentences-wordnet_sense_id_example[4]_embeddings.json"


output_file = f"../data/outputs/annotation_phase_2/eng_combined_data.json"

In [10]:
with open(historical_sample_file, "r") as f:
    historical_sample = json.load(f)

    complete_historical = []

    for lemma in historical_sample:
        for usage in historical_sample[lemma]:
            if usage['empty_synsets'] == 0:
                complete_historical.append({
                    "lemma": lemma,
                    "sentence": usage['usage']['sentence'],
                    "target": usage['usage']['target'],
                    "highest_similarity": usage['usage']['closest'][0][1],
                    "corpus_id": f"ccoha1"
                })

complete_historical = pd.DataFrame(complete_historical)
complete_historical = complete_historical.drop_duplicates(subset=['sentence'])
complete_historical = complete_historical.sort_values(by=['highest_similarity'])

print(complete_historical.shape)
display(complete_historical.head())

historical_dict = complete_historical.to_dict(orient='records')
display(historical_dict[0])

(165, 5)


,lemma,sentence,target,highest_similarity,corpus_id
0,unused,"During my voyage , affrighted by the dangers w...","[93, 99]",0.223719,ccoha1
49,trample,"It were less cruel for me to have slain her , ...","[168, 175]",0.251587,ccoha1
21,strongly,When the wind blows strongly from the north an...,"[20, 28]",0.257914,ccoha1
26,warmly,"Besides , had it been warmly espoused at the f...","[22, 28]",0.267301,ccoha1
149,shirk,"to the river and washed , first for , it was i...","[282, 287]",0.267521,ccoha1


{'lemma': 'unused',
 'sentence': 'During my voyage , affrighted by the dangers which surrounded me , and to which I was wholly unused , I heartily repented of my resolution but now , methinks , I have reason to rejoice at my perseverance .',
 'target': [93, 99],
 'highest_similarity': 0.2237192327980831,
 'corpus_id': 'ccoha1'}

In [11]:
with open(modern_sample_file, "r") as f:
    modern_sample = json.load(f)

    complete_modern = []

    for lemma in modern_sample:
        for usage in modern_sample[lemma]:
            if usage['empty_synsets'] == 0:
                complete_modern.append({
                    "lemma": lemma,
                    "sentence": usage['usage']['sentence'],
                    "target": usage['usage']['target'],
                    "highest_similarity": usage['usage']['closest'][0][1],
                    "corpus_id": f"leipzig_eng_news"
                })

complete_modern = pd.DataFrame(complete_modern)
complete_modern = complete_modern.drop_duplicates(subset=['sentence'])
complete_modern = complete_modern.sort_values(by=['highest_similarity'])

print(complete_modern.shape)
display(complete_modern.head())

modern_dict = complete_modern.to_dict(orient='records')
display(modern_dict[0])

(259, 5)


,lemma,sentence,target,highest_similarity,corpus_id
70,qualification,Slovan Bratislava's Champions League qualifica...,"[37, 50]",0.173049,leipzig_eng_news
68,qualification,Streichs team is currently eighth in the 18tea...,"[120, 133]",0.188805,leipzig_eng_news
67,qualification,"But they have fallen away badly, winning only ...","[144, 157]",0.192252,leipzig_eng_news
66,qualification,The bid for Aberdeens first Scottish Cup since...,"[65, 78]",0.194470,leipzig_eng_news
195,wage,Davis gained fame as a state legislator by wag...,"[43, 47]",0.194945,leipzig_eng_news


{'lemma': 'qualification',
 'sentence': "Slovan Bratislava's Champions League qualification tie against Faroe Islands side KI Klaksvik has been cancelled after a player from the Slovakian club tested positive for COVID19, European soccer's governing body UEFA said on Saturday.",
 'target': [37, 50],
 'highest_similarity': 0.17304900400543594,
 'corpus_id': 'leipzig_eng_news'}

In [12]:
# combine both datasets
combined_data = historical_dict + modern_dict
combined_data = pd.DataFrame(combined_data)
combined_data = combined_data.drop_duplicates(subset=['lemma', 'sentence'])
combined_data = combined_data.sort_values(by=['corpus_id', 'highest_similarity'])

combined_data.to_json(output_file, orient='records', indent=4)

print(combined_data.shape)
display(combined_data.head())

(424, 5)


,lemma,sentence,target,highest_similarity,corpus_id
0,unused,"During my voyage , affrighted by the dangers w...","[93, 99]",0.223719,ccoha1
1,trample,"It were less cruel for me to have slain her , ...","[168, 175]",0.251587,ccoha1
2,strongly,When the wind blows strongly from the north an...,"[20, 28]",0.257914,ccoha1
3,warmly,"Besides , had it been warmly espoused at the f...","[22, 28]",0.267301,ccoha1
4,shirk,"to the river and washed , first for , it was i...","[282, 287]",0.267521,ccoha1
